In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

### Prepare input file

In [3]:
from decouple import config
from huggingface_hub import AsyncInferenceClient


HUGGINGFACE_TOKEN = config('HUGGINGFACE_TOKEN')

MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'

client = AsyncInferenceClient(
    model=MODEL_HUB_ID,
    provider='hf-inference',
    timeout=None,
    api_key=HUGGINGFACE_TOKEN,
)

In [3]:
from math_rag.application.base.assistants import BaseAssistantInput, BaseAssistantOutput


class SomeInput(BaseAssistantInput):
    pass


class SomeOutput(BaseAssistantOutput):
    result: int

In [4]:
import json

from pathlib import Path

from math_rag.application.models.inference import (
    LLMBatchRequest,
    LLMBatchResult,
    LLMConversation,
    LLMMessage,
    LLMParams,
    LLMRequest,
)
from math_rag.infrastructure.mappings.inference.huggingface import (
    LLMRequestMapping,
    LLMResponseListMapping,
)


MODEL_HUB_ID = 'microsoft/Phi-3-mini-4k-instruct'
some_input = SomeInput()

# request = LLMRequest(
#     conversation=LLMConversation(
#         messages=[
#             LLMMessage(role='system', content='You are a helpful assistant.'),
#             LLMMessage(role='user', content='what is 2+2'),
#         ]
#     ),
#     params=LLMParams(
#         model=MODEL_HUB_ID,
#         temperature=0,
#         response_type=SomeOutput,
#         max_completion_tokens=10,
#         metadata={'input_id': str(some_input.id)},
#     ),
# )

# batch_request: LLMBatchRequest = LLMBatchRequest(requests=[request])

_requests = [
    LLMRequest(
        conversation=LLMConversation(
            messages=[
                LLMMessage(role='system', content='You are a helpful assistant.'),
                LLMMessage(role='user', content=f'what is {i}+2'),
            ]
        ),
        params=LLMParams(
            model=MODEL_HUB_ID,
            temperature=0,
            response_type=SomeOutput,
            max_completion_tokens=10,
            metadata={'input_id': str(some_input.id)},
        ),
    )
    for i in range(200)
]

batch_request: LLMBatchRequest = LLMBatchRequest(requests=_requests)

In [5]:
requests = [LLMRequestMapping.to_target(request) for request in batch_request.requests]
lines = [json.dumps(request, separators=(',', ':')) for request in requests]
jsonl_str = '\n'.join(lines)
jsonl_bytes = jsonl_str.encode('utf-8')

In [7]:
request_dict = json.loads(lines[0])

In [8]:
result = await client.chat_completion(**request_dict)
result

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='{ "result": 4 }', tool_call_id=None, tool_calls=None), logprobs=None)], created=1744125157, id='', model='microsoft/Phi-3-mini-4k-instruct', system_fingerprint='3.2.1-native', usage=ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=17, total_tokens=26), object='chat.completion')

In [40]:
response_list = LLMResponseListMapping.to_source(
    result,
    request_id=request.id,
    input_id=request_dict['extra_body']['input_id'],
    response_type=SomeOutput,
)
response_list

LLMResponseList(id=UUID('f7fd3b29-2772-468d-831f-b160bf1dfaa0'), request_id=UUID('cb02d5a3-bed8-4109-b06c-04d585081ef3'), responses=[LLMResponse(id=UUID('3976268b-adbf-494a-b88b-fda24781d477'), content=BoundAssistantOutput(id=UUID('bd7fdedd-f32a-4d7f-845d-0a21413828b6'), input_id=UUID('c882baa3-2047-4f65-b8df-bd176bba3b2c'), result=4), logprobs=None)])

In [ ]:
input_file_path = Path(f'.tmp/input_{batch_request.id}.jsonl')

with open(input_file_path, 'w') as input_file:
    for line in lines:
        input_file.write(line + '\n')

### Apptainer

In [5]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

tgi_batch_llm = infrastructure_container.tgi_batch_llm()

In [6]:
await tgi_batch_llm.init_resources()

2025-04-29 12:38:28,880 - INFO - Host canonicalization disabled
2025-04-29 12:38:28,880 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-04-29 12:38:28,947 - INFO - [conn=0] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-04-29 12:38:28,947 - INFO - [conn=0]   Local address: 172.22.0.10, port 39124
2025-04-29 12:38:28,948 - INFO - [conn=0]   Peer address: 161.53.2.37, port 22
2025-04-29 12:38:29,026 - INFO - [conn=0] Beginning auth for user lpanic
2025-04-29 12:38:29,305 - INFO - [conn=0] Auth for user lpanic succeeded
2025-04-29 12:38:29,306 - INFO - [conn=0, chan=0] Requesting new SSH session
2025-04-29 12:38:29,476 - INFO - [conn=0, chan=0]   Command: mkdir -p tgi_default_root
2025-04-29 12:38:29,626 - INFO - [conn=0, chan=0] Received exit status 0
2025-04-29 12:38:29,627 - INFO - [conn=0, chan=0] Received channel close
2025-04-29 12:38:29,627 - INFO - [conn=0, chan=0] Channel closed
2025-04-29 12:38:29,627 - INFO - Command `mkdir -p tgi_default_

In [7]:
res = await tgi_batch_llm.batch_generate(
    batch_request=batch_request,
    response_type=SomeOutput,
    poll_interval=3 * 60,
    max_tokens_per_day=None,
    max_num_retries=0,
)
res

2025-04-29 12:49:36,158 - INFO - Host canonicalization disabled
2025-04-29 12:49:36,159 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-04-29 12:49:36,170 - INFO - [conn=22] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-04-29 12:49:36,170 - INFO - [conn=22]   Local address: 172.22.0.10, port 49214
2025-04-29 12:49:36,170 - INFO - [conn=22]   Peer address: 161.53.2.37, port 22
2025-04-29 12:49:36,221 - INFO - [conn=22] Beginning auth for user lpanic
2025-04-29 12:49:36,505 - INFO - [conn=22] Auth for user lpanic succeeded
2025-04-29 12:49:36,509 - INFO - [conn=22, chan=0] Requesting new SSH session
2025-04-29 12:49:36,659 - INFO - [conn=22, chan=0]   Subsystem: sftp
2025-04-29 12:49:36,670 - INFO - [conn=22, chan=0] Starting SFTP client
2025-04-29 12:49:36,936 - INFO - [conn=22, chan=0] Received exit status 0
2025-04-29 12:49:36,938 - INFO - [conn=22, chan=0] Received channel close
2025-04-29 12:49:36,940 - INFO - [conn=22, chan=0] SFTP client exi

LLMBatchResult(id=UUID('05e9aba1-e2d9-42e3-b833-31909ab4c576'), batch_request_id=UUID('b1916016-9185-4048-bb64-c16af84e9602'), response_lists=[LLMResponseList(id=UUID('2464e16d-c69d-4187-bf1e-ba8fcc77e49f'), request_id=UUID('3f439f95-b969-4403-81dc-b4d84ba0f6ad'), responses=[LLMResponse(id=UUID('6c8eec46-05be-476b-b6d1-cbbc467021b1'), content=BoundAssistantOutput(id=UUID('28f133f6-63da-44fc-abd3-cd5a84dd6b2b'), input_id=UUID('da53211d-e865-48fb-b07e-0e5ecced5f89'), result=72), logprobs=None)]), LLMResponseList(id=UUID('144af837-08a1-48fa-a2a7-ec0eebe76067'), request_id=UUID('aead3f9b-4226-4cbd-90b7-3b3554e6f10b'), responses=[LLMResponse(id=UUID('c8eeb690-3310-4f08-b804-3fa72d6234b1'), content=BoundAssistantOutput(id=UUID('d431bcb9-8383-4494-9658-f2aad30829c2'), input_id=UUID('da53211d-e865-48fb-b07e-0e5ecced5f89'), result=99), logprobs=None)]), LLMResponseList(id=UUID('19780f6a-10c5-4ccd-8f91-b0a84e34d247'), request_id=UUID('69a35506-a3cb-49e8-914c-9f2b5eef8d9d'), responses=[LLMRespons

In [9]:
from pathlib import Path


sftp_client = infrastructure_container.sftp_client()

# tar -czvf snapshot.tar.gz data

# TODO download dir
await sftp_client.download(
    Path('tgi_default_root/snapshot.tar.gz'), Path('../.tmp/snapshot.tar.gz')
)

# tar -xzvf snapshot.tar.gz

2025-04-29 13:08:38,862 - INFO - Host canonicalization disabled
2025-04-29 13:08:38,862 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-04-29 13:08:38,875 - INFO - [conn=38] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-04-29 13:08:38,876 - INFO - [conn=38]   Local address: 172.22.0.10, port 43108
2025-04-29 13:08:38,876 - INFO - [conn=38]   Peer address: 161.53.2.37, port 22
2025-04-29 13:08:38,915 - INFO - [conn=38] Beginning auth for user lpanic
2025-04-29 13:08:39,184 - INFO - [conn=38] Auth for user lpanic succeeded
2025-04-29 13:08:39,187 - INFO - [conn=38, chan=0] Requesting new SSH session
2025-04-29 13:08:39,325 - INFO - [conn=38, chan=0]   Subsystem: sftp
2025-04-29 13:08:39,335 - INFO - [conn=38, chan=0] Starting SFTP client
2025-04-29 13:08:39,562 - INFO - [conn=38, chan=0] Received exit status 0
2025-04-29 13:08:39,563 - INFO - [conn=38, chan=0] Received channel close
2025-04-29 13:08:39,563 - INFO - [conn=38, chan=0] SFTP client exi

In [ ]:
from os import environ

from decouple import config


environ['PBS_O_WORKDIR'] = '../.tmp'
environ['TGI_API_KEY'] = config('HUGGINGFACE_TOKEN')
environ['MODEL_HUB_ID'] = 'microsoft/Phi-3-mini-4k-instruct'

%run ../assets/hpc/hf/tgi/tgi_client.py